# Finding putative metabolite interactions

We will employ [SMETANA]() through its [Python implementation]() to find putative metabolic interactions among the set of models in our use case: a phycosphere microbiome composed of the diatom: Pseudo-nitzschia sp., and the bacterial partners: Sulfitobacter sp., Alteromonas sp., Marinobacter sp., and Polaribacter sp.

To this end, we will simulate a marine medium containing only inorganic componets, as vitamins and cofactors will be synthesized by the bacterial partners, while the diatom will provide the carbon source.

## Which bacterial partners are producing Pseudo-nitzschia essential cofactors?

* Adenosylcobalamin (adocbl): _Sulfitobacter sp._
* Folate (fol): _?_
* Thiamin (thm): _?_ https://www.frontiersin.org/articles/10.3389/fmars.2020.606342/full


It seems like none of the reconstructed GEMs are producing thiamin and folate per se, but rather importing. Is this a limitation of the universal model or are these species really auxotrophs for them?

| Model | B12 (adocbl) | B1 (thm) | Folate |
|-------|----------|----------|--------|
| Sulfitobacter   | +   | -   | -   |
| Alteromonas   | -   | -   | -   |
| Marinobacter   | -   | -   | -   |
| Polaribacter   | -   | -   | -   |

In [10]:
%%bash

smetana \
    ../results/models/*.xml \
    -m M9[marine] \
    --mediadb ../data/marine_media/media_db.tsv \
    -o ../results/pseudo_nitzschia_interactions \
    --solver gurobi \
    --molweight --detailed
    # --exclude ../data/compounds/inorganic.txt \

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-05


In [11]:
import pandas as pd


df = pd.read_csv("../results/pseudo_nitzschia_interactions_detailed.tsv", sep="\t")
df

,community,medium,receiver,donor,compound,scs,mus,mps,smetana


## Which compounds can PS secrete?

In [53]:
import cobra 

models = {}
models["ps"] = cobra.io.read_sbml_model("../results/models/MAG_00212_pseudonitzschia_photoeuk_noSK.xml")

In [5]:
models["ps"].reactions.get_by_id("EX_tsul_e")
models["ps"].metabolites.get_by_id("CPD-15199_c")

Metabolite identifier,CPD-15199_c
Name,5-deoxy-α-ribose 1-phosphate
Memory address,0x7fc6c0ec93d0
Formula,C5H9O7P
Compartment,c
In 2 reaction(s),"RXN-14304, DM_CPD-15199_c"


In [50]:
from reconstruction import get_organic_exchanges
from cobra.flux_analysis.variability import flux_variability_analysis, find_blocked_reactions


carbon_exchanges = get_organic_exchanges(models["ps"])
fva_ex = flux_variability_analysis(models["ps"], reaction_list=carbon_exchanges, loopless=False, fraction_of_optimum=0.1)

In [51]:
fva_ex[fva_ex.maximum > 0]

,minimum,maximum
EX_glyclt_e,0.0,38.358169
EX_glc_e,0.0,18.674589
EX_tcynt_e,0.0,23.014901
EX_lac_d_e,0.0,10.000000
EX_etoh_e,0.0,74.698354
...,...,...
EX_ser_L_e,0.0,10.000000
EX_13dpg_e,0.0,10.000000
EX_2aobut_e,0.0,10.000000
EX_ala_B_e,0.0,10.000000


## What about exports in bacterial partners?

In [60]:
models["sulfito"] = cobra.io.read_sbml_model("../results/models/TARA_ARC_108_MAG_00083.genepred.xml")


carbon_exchanges = models["sulfito"].exchanges #get_organic_exchanges(models["sulfito"])

In [59]:
models["sulfito"].metabolites.get_by_id("ascb__L_e")

Metabolite identifier,ascb__L_e
Name,L-Ascorbate
Memory address,0x7fc6ff62c290
Formula,None
Compartment,e
In 2 reaction(s),"EX_ascb__L_e, ASCBpts"


In [61]:
fva_ex = flux_variability_analysis(models["sulfito"], reaction_list=carbon_exchanges, loopless=False, fraction_of_optimum=0.1)

fva_ex[fva_ex.maximum > 0]

,minimum,maximum
EX_ala_B_e,0.0,711.776651
EX_colipa_e,0.0,11.525924
EX_enlipa_e,0.0,28.631432
EX_fald_e,0.0,1000.000000
EX_indole_e,0.0,28.851127
EX_o16a4colipa_e,0.0,6.769035
EX_pheme_e,0.0,9.953859
EX_2mbtoh_e,0.0,301.048641
EX_iamoh_e,0.0,246.542125
EX_ibutoh_e,0.0,297.966954
